In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import ast
import wfdb
from tensorflow import keras
from tensorflow import lite
from sklearn.preprocessing import MultiLabelBinarizer
print("Done")

Done


In [ ]:
def load_raw_data(df, sampling_rate, path):
    if sampling_rate == 100:
        data = [wfdb.rdsamp(path+f) for f in df.filename_lr]
    else:
        data = [wfdb.rdsamp(path+f) for f in df.filename_hr]
    data = np.array([signal for signal, meta in data])
    return data

path = '../../../Datasets/ptb-xl-a-large-publicly-available-electrocardiography-dataset-1.0.1\\'
sampling_rate=100
calssificatin_type = "superclasses"    #{"binary","superclasses","subclasses"}

lead_types={"lead-I":[1,2,3,4,5,6,7,8,9,10,11], "bipolar-limb":[3,4,5,6,7,8,9,10,11] , "unipolar-limb":[0,1,2,6,7,8,9,10,11], "limb-leads":[6,7,8,9,10,11] , "precordial-leads":[0,1,2,3,4,5],"all-lead":[]}
lead_name="all-lead"

# load and convert annotation data
Y = pd.read_csv(path+'ptbxl_database.csv', index_col='ecg_id')
Y.scp_codes = Y.scp_codes.apply(lambda x: ast.literal_eval(x))

# Load raw signal data
X = load_raw_data(Y, sampling_rate, path)

# Load scp_statements.csv for diagnostic aggregation
agg_df = pd.read_csv(path+'scp_statements.csv', index_col=0)
agg_df = agg_df[agg_df.diagnostic == 1]

def aggregate_superclass_diagnostic(y_dic):
    tmp = []
    for key in y_dic.keys():
        if key in agg_df.index:
            tmp.append(agg_df.loc[key].diagnostic_class)
    return list(set(tmp))

def aggregate_subclass_diagnostic(y_dic):
    tmp = []
    for key in y_dic.keys():
        if key in agg_df.index:
            tmp.append(agg_df.loc[key].diagnostic_subclass)
    ret = list(set(tmp))
    return ret

if calssificatin_type == "superclasses":
    Y['diagnostic_superclass'] = Y.scp_codes.apply(aggregate_superclass_diagnostic)
else:
    Y['diagnostic_subclass'] = Y.scp_codes.apply(aggregate_subclass_diagnostic)
    
# Ensure y_train is correctly transformed
mlb = MultiLabelBinarizer(classes=['CD', 'HYP', 'MI', 'NORM', 'STTC'])
Y = mlb.fit_transform(Y.diagnostic_superclass)

print("Done")

In [ ]:
X1 = np.array(X)
X2 = []
for i in range(len(X1)):
    X2.append(X1[i].transpose())

In [ ]:
# import train_test_split from sklearn.model_selection for splitting the dataset
from sklearn.model_selection import train_test_split

# splitting the dataset into test and train dataset
X_train, X_test, y_train, y_test = train_test_split(X2, Y, test_size=0.2, random_state=42)

# print lenght of thest and train dataset
print("Training data:")
print(len(X_train))
print("Training labels:")
print(len(y_train))
print("Testing data:")
print(len(X_test))
print("Testing labels:")
print(len(y_test))

In [5]:
# testing the models with given data index (i)

i = 1

for j in range(12):
    model = tf.keras.models.load_model(f'trial_2_{j+1}_model.h5')
    input_data = X_test[i-1][j]
    input_data = input_data.reshape(1, 1000, 1) 
    predictions = model.predict(input_data)
    print(predictions)
    print(f"Prediction : {[1 if x > 0.5 else 0 for x in predictions[0]]}")
    print(f"True : {y_test[i-1]}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 392ms/step
[[0.72913176 0.15022688 0.37553376 0.09492628 0.13110994]]
Prediction : [1, 0, 0, 0, 0]
True : [1 0 0 0 0]


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 475ms/step
[[0.26098523 0.09586097 0.2846017  0.4275405  0.15254761]]
Prediction : [0, 0, 0, 0, 0]
True : [1 0 0 0 0]


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 222ms/step
[[0.1121065  0.05071908 0.08830016 0.67902625 0.17704579]]
Prediction : [0, 0, 0, 1, 0]
True : [1 0 0 0 0]


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 173ms/step


[[0.8469813  0.11002915 0.5514455  0.02488077 0.11463881]]
Prediction : [1, 0, 1, 0, 0]
True : [1 0 0 0 0]


...

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 221ms/step
[[0.7694133  0.21265079 0.4889134  0.01387787 0.10201441]]
Prediction : [1, 0, 0, 0, 0]
True : [1 0 0 0 0]


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 259ms/step
[[0.14929284 0.24329165 0.18547256 0.18934555 0.35345677]]
Prediction : [0, 0, 0, 0, 0]
True : [1 0 0 0 0]


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 228ms/step
[[0.26354086 0.08279222 0.24480692 0.16065939 0.47710305]]
Prediction : [0, 0, 0, 0, 0]
True : [1 0 0 0 0]


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 233ms/step
[[0.46568203 0.12421826 0.38397053 0.2488414  0.28161696]]
Prediction : [0, 0, 0, 0, 0]
True : [1 0 0 0 0]


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 213ms/step
[[0.42542085 0.09477528 0.3037608  0.35625517 0.14786705]]
Prediction : [0, 0, 0, 0, 0]
True : [1 0 0 0 0]


In [6]:
# # Save test 12 lead data as CSV file with transposed data

# for i in range(len(X_test)):
#     df = pd.DataFrame(X_test[i])
#     df.to_csv(f'../12_lead_test_data/12_lead_ecg_test_{i}.csv', index = False)
# print("Done")

In [7]:
# data visualization

# import matplotlib.pyplot as plt
# for i in range(5):
#     plt.plot(X_train[i])
#     plt.title(y_train[i])
#     plt.show()